In [ ]:
%matplotlib inline

# BiLevel-based DOE on the Sobieski SSBJ test case.


In [ ]:
from __future__ import annotations

from copy import deepcopy
from os import name as os_name

from gemseo import create_discipline
from gemseo import create_scenario
from gemseo import execute_post
from gemseo.problems.mdo.sobieski.core.design_space import SobieskiDesignSpace

## Instantiate the  disciplines
First, we instantiate the four disciplines of the use case:
[SobieskiPropulsion][gemseo.problems.mdo.sobieski.disciplines.SobieskiPropulsion],
[SobieskiAerodynamics][gemseo.problems.mdo.sobieski.disciplines.SobieskiAerodynamics],
[SobieskiMission][gemseo.problems.mdo.sobieski.disciplines.SobieskiMission]
and [SobieskiStructure][gemseo.problems.mdo.sobieski.disciplines.SobieskiStructure].



In [ ]:
propu, aero, mission, struct = create_discipline([
    "SobieskiPropulsion",
    "SobieskiAerodynamics",
    "SobieskiMission",
    "SobieskiStructure",
])

## Build, execute and post-process the scenario
Then, we build the scenario which links the disciplines
with the formulation and the optimization algorithm. Here, we use the
[BiLevel][gemseo.formulations.bilevel.BiLevel] formulation. We tell the scenario to minimize -y_4
instead of minimizing y_4 (range), which is the default option.

We need to define the design space.



In [ ]:
design_space = SobieskiDesignSpace()

Then, we build a sub-scenario for each strongly coupled disciplines.



### Build a sub-scenario for Propulsion
This sub-scenario will minimize SFC.



In [ ]:
sc_prop = create_scenario(
    propu,
    "y_34",
    design_space.filter("x_3", copy=True),
    name="PropulsionScenario",
    formulation_name="DisciplinaryOpt",
)

### Build a sub-scenario for Aerodynamics
This sub-scenario will minimize L/D.



In [ ]:
sc_aero = create_scenario(
    aero,
    "y_24",
    design_space.filter("x_2", copy=True),
    name="AerodynamicsScenario",
    maximize_objective=True,
    formulation_name="DisciplinaryOpt",
)

### Build a sub-scenario for Structure
This sub-scenario will maximize
log(aircraft total weight / (aircraft total weight - fuel weight)).



In [ ]:
sc_str = create_scenario(
    struct,
    "y_11",
    deepcopy(design_space).filter("x_1"),
    name="StructureScenario",
    maximize_objective=True,
    formulation_name="DisciplinaryOpt",
)

### Build a scenario for Mission
This scenario is based on the three previous sub-scenarios and on the
Mission and aims to maximize the range (Breguet).



In [ ]:
sub_disciplines = [sc_prop, sc_aero, sc_str, mission]
system_scenario = create_scenario(
    sub_disciplines,
    "y_4",
    design_space.filter("x_shared", copy=True),
    parallel_scenarios=False,
    reset_x0_before_opt=True,
    scenario_type="DOE",
    formulation_name="BiLevel",
    save_opt_history="True",
    naming="UUID",
)

!!! note

      Setting `reset_x0_before_opt=True` is mandatory when doing a DOE
      in parallel. If we want reproducible results, don't reuse previous xopt.



!!! tip

      When running BiLevel scenarios, it is interesting to access the optimization
      history of the sub-scenarios for each system iteration. By default, the setting
      `keep_opt_history` is set to `True`. This allows you to store in memory the
      databases of the sub-scenarios (see the last section of this example for more
      details).
      In some cases, storing the databases in memory can take up too much space and cause
      performance issues. In these cases, set `keep_opt_history=False` and save the
      databases to the disk using `save_opt_history=True`. If your sub-scenarios are
      running in parallel, and you are saving the optimization histories to the disk, set
      the `naming` setting to `"UUID"`, which is multiprocessing-safe.
      The setting `keep_opt_history` will not work if the sub-scenarios are running in
      parallel because the databases are not copied from the sub-processes to the main
      process. In this case you shall always save the optimization history to the disk.



In [ ]:
system_scenario.formulation.mda1.warm_start = False
system_scenario.formulation.mda2.warm_start = False

!!! note

      This is mandatory when doing a DOE in parallel if we want always exactly
      the same results, don't warm start mda1 to have exactly the same
      process whatever the execution order and process dispatch.



In [ ]:
for sub_sc in sub_disciplines[0:3]:
    sub_sc.set_algorithm(algo_name="L-BFGS-B", max_iter=20)

### Visualize the XDSM
Generate the XDSM on the fly:

- `log_workflow_status=True` will log the status of the workflow  in the console,
- `save_html` (default `True`) will generate a self-contained HTML file,
  that can be automatically opened using `show_html=True`.



In [ ]:
system_scenario.xdsmize(save_html=False)

### Multiprocessing
It is possible to run a DOE in parallel using multiprocessing, in order to do
this, we specify the number of processes to be used for the computation of
the samples.



!!! warning
      The multiprocessing option has some limitations on Windows.
      Due to problems with building the documentation, we disable it in this example.
      The features [MemoryFullCache][gemseo.caches.memory_full_cache.MemoryFullCache] and [HDF5Cache][gemseo.caches.hdf5_cache.HDF5Cache] are not
      available for multiprocessing on Windows.
      As an alternative, we recommend the method
      [set_optimization_history_backup()][gemseo.scenarios.doe_scenario.DOEScenario.set_optimization_history_backup].



In [ ]:
n_processes = 1 if os_name == "nt" else 4
system_scenario.execute(algo_name="PYDOE_LHS", n_samples=30, n_processes=n_processes)

system_scenario.print_execution_metrics()

!!! warning
      On Windows, the progress bar may show duplicated instances during the
      initialization of each subprocess. In some cases it may also print the
      conclusion of an iteration ahead of another one that was concluded first.
      This is a consequence of the pickling process and does not affect the
      computations of the scenario.



### Exporting the problem data.
After the execution of the scenario, you may want to export your data to use it
elsewhere. The method [to_dataset()][gemseo.scenarios.base_scenario.BaseScenario.to_dataset] will allow you to export
your results to a [Dataset][gemseo.datasets.dataset.Dataset], the basic GEMSEO class to store data.



In [ ]:
dataset = system_scenario.to_dataset("a_name_for_my_dataset")

### Plot the optimization history view



In [ ]:
system_scenario.post_process(post_name="OptHistoryView", save=False, show=True)

### Plot the scatter matrix



In [ ]:
system_scenario.post_process(
    post_name="ScatterPlotMatrix",
    variable_names=["y_4", "x_shared"],
    save=False,
    show=True,
)

### Plot parallel coordinates



In [ ]:
system_scenario.post_process(post_name="ParallelCoordinates", save=False, show=True)

### Plot correlations



In [ ]:
system_scenario.post_process(post_name="Correlations", save=False, show=True)

### Plot the structure optimization histories of the 2 first iterations
The code below will not work if you ran the system scenario with `n_processes` > 1.
Indeed, parallel execution of sub-scenarios prevents us to save the databases from
each sub-process to the main process. If you ran the system scenario with many
processes, you can still save the databases to the disk with
`save_opt_history=True` and `naming="UUID"`. Refer to the formulation settings for
more information.



In [ ]:
struct_databases = system_scenario.formulation.scenario_adapters[2].databases
for database in struct_databases[:2]:
    opt_problem = deepcopy(sc_str.formulation.optimization_problem)
    opt_problem.database = database
    execute_post(opt_problem, post_name="OptHistoryView", save=False, show=True)